# Multinomial logit

This example shows a simple multinomial logit model using the [LPMC dataset](link).

In [ ]:
#import packages
import pandas as pd
from pycmtensor.dataset import Dataset
from pycmtensor.expressions import Beta
from pycmtensor.models import MNL


# load data
lpmc = pd.read_csv("../../data/lpmc.dat", sep="\t")
ds = Dataset(df=lpmc, choice="travel_mode")
ds.split(0.8)


# Beta parameters
asc_walk = Beta("asc_walk", 0.0, None, None, 1)
asc_cycle = Beta("asc_cycle", 0.0, None, None, 0)
asc_pt = Beta("asc_pt", 0.0, None, None, 0)
asc_drive = Beta("asc_drive", 0.0, None, None, 0)
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
s_time = Beta("s_time", 0.5, None, None, 0)
b_purpose = Beta("b_purpose", 0.0, None, None, 0)
b_licence = Beta("b_licence", 0.0, None, None, 0)
b_car_own = Beta("b_car_own", 0.0, None, None, 0)


# composite variables
dur_pt = ds["dur_pt_rail"] + ds["dur_pt_bus"] + ds["dur_pt_int"]
cost_drive = ds["cost_driving_fuel"] + ds["cost_driving_ccharge"]


# utility functions
U_walk = asc_walk + b_time * ds["dur_walking"]
U_cycle = asc_cycle + b_time * ds["dur_cycling"] 
U_pt = asc_pt + b_time * dur_pt + b_cost * ds["cost_transit"]
U_drive = (asc_drive + b_time * ds["dur_driving"] + b_cost * cost_drive 
        + b_licence * ds["driving_license"] + b_purpose * ds["purpose"] 
		+ b_car_own * ds["car_ownership"])

U = [U_walk, U_cycle, U_pt, U_drive]


# the choice model is MNL
mymodel = MNL(ds, locals(), U)


# estimate the model
from pycmtensor.models import train
train(mymodel, ds)

The following code displays the results of the model estimation

In [ ]:
print(mymodel.results.beta_statistics())
print(mymodel.results.model_statistics())
print(mymodel.results.benchmark())

The following code displays the model correlation and robust correlation matrices

In [ ]:
mymodel.results.model_correlation_matrix()

In [ ]:
mymodel.results.model_robust_correlation_matrix()

The following code displays the model prediction on the validation dataset

In [ ]:
prob = mymodel.predict(ds, return_probabilities=True)
pd.DataFrame(prob)

In [ ]:
choice = mymodel.predict(ds, return_probabilities=False)
pd.DataFrame(choice)

The following code displays the model elasticities (w.r.t to choice 1)

In [ ]:
elas = mymodel.elasticities(ds, wrt_choice=1)
pd.DataFrame(elas)